#### Training using Yolov8

First step it is to create our project

In [ ]:
import mlrun

project = mlrun.get_or_create_project('yolotraining', context="./", user_project=True)

Create Secrets

In [ ]:
secrets = {"MINIO_URL": "", "MINIO_AK": "", "MINIO_SK": ""}

In [ ]:
project.set_secrets(secrets=secrets, provider="kubernetes")

For this project we created a simple traning function, in the job_func file we have the following traning function:

```python
import io
import os
from datetime import datetime

import minio
from ultralytics import YOLO


# Recursively traverse the local folder and upload files.
def upload_files(folderPath, objectPrefix, minioClient, bucketName):
    # add date
    now = str(datetime.now())
    for root, _, files in os.walk(folderPath):
        for fileName in files:
            filePath = os.path.join(root, fileName)
            relativePath = os.path.relpath(filePath, folderPath)
            objectPath = os.path.join(now, objectPrefix, relativePath)

            minioClient.fput_object(bucketName, objectPath, filePath)


def yolo_train(context, params: dict):
    model = YOLO(params["base_model"])
    results = model.train(
        data=params["data"], epochs=params["epochs"], imgsz=params["imgsz"]
    )

    minioClient = minio.Minio(
        os.environ.get("MLRUN_K8S_SECRET__MINIO_URL"),
        access_key=os.environ.get("MLRUN_K8S_SECRET__MINIO_AK"),
        secret_key=os.environ.get("MLRUN_K8S_SECRET__MINIO_SK"),
    )

    # Set the bucket and folder names.
    bucketName = "models"
    folderName = "runs/detect/train"

    # Create the folder in Minio if it doesn't already exist.
    if not minioClient.bucket_exists(bucketName):
        minioClient.make_bucket(bucketName)

    # Upload files from the specified folder.
    upload_files(folderName, folderName, minioClient, bucketName)

```

Then we set our new function based on our file `job_func.py` as a `job` and set the hadler to be the name of our training function `yolo_train()`. For the image we are going the use the one recomended for Deep Learning algoritms. If you have a GPU change the image to `mlrun/mlrun-gpu`

In [ ]:
fn = project.set_function(
    name="python", func="job_func.py",  kind="job",
    image="smartcommunitylab/yolov5-minio:tst", handler="yolo_train"
)

Next, we specify the build commands for our image, installing minio and the ultralytics requirements

In [ ]:
fn.spec.build.commands = [
    "apt-get update && apt-get install -y cmake gcc g++ libglu1-mesa-dev",
    "python -m pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt",
    "python -m pip install minio"
]

building the image should take some minutes until all the packages are instaled in the image

In [ ]:
project.build_function(fn)

Finally we can proceed to do the training were we pass to our function the params of training.

In [ ]:
from kubernetes import client
tol = [
    client.V1Toleration(
        key='nvidia.com/gpu',
        operator='Equal',
        value='v100',\
        effect='NoSchedule',
    )
]
fn.with_node_selection(tolerations=tol)
fn.with_limits(gpus=1,mem="70G")

In [ ]:
params = {"data": 'coco128.yaml', "epochs": 1, "imgsz":640, "base_model": "yolov8n.pt", "workers":0, "device":0}
fn.run(params={'params': params})

The training files are available on Minio, under the bucket `models`, organized by date and time. The best model is `best.pt` in the path  `<date>/runs/detect/train/weights/best.pt`.